In [1]:
import pandas as pd
import seaborn as sns
df=pd.read_csv('/Users/rishilboddula/Desktop/MLOPS/Sentiment-Analysis-MLOPS/archive (13)/training.1600000.processed.noemoticon.csv')
df.head()
df.tail()

,polarity of tweet�,id of the tweet,date of the tweet,query,user,text of the tweet�
1048567,4,1960186342,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Madelinedugganx,My GrandMa is making Dinenr with my Mum
1048568,4,1960186409,Fri May 29 07:33:43 PDT 2009,NO_QUERY,OffRoad_Dude,Mid-morning snack time... A bowl of cheese noo...
1048569,4,1960186429,Fri May 29 07:33:44 PDT 2009,NO_QUERY,Falchion,@ShaDeLa same here say it like from the Termi...
1048570,4,1960186445,Fri May 29 07:33:44 PDT 2009,NO_QUERY,jonasobsessedx,@DestinyHope92 im great thaanks wbuu?
1048571,4,1960186607,Fri May 29 07:33:45 PDT 2009,NO_QUERY,sugababez,cant wait til her date this weekend


In [13]:
import pandas as pd
import numpy as np
import re
import logging
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Tuple
from typing_extensions import Annotated
import pickle

'''



Now in the below steps for feautre engineering we do the following
           1-We put the text to lower
           2- remove the user accounts in text like @elon_musk or @account
           3-we remove any urls
           4- we remove all puncuation marks and numbers
           5-we put a sentence in an array, like ex-
           i/p-I love Ferrari 
           o/p-['I','love','Ferrari;]
           6- we remove all the the common words and convert most words to root words
           7- WE can use WordNetLemmatizer() instead of port stemmer and that is efficient because it
           converts the word to theier dictionary roots where as POrtstemmer uses crude rules.But WordNetLemmatizer() is
           very slow.For this project we use Portstemmer
           ex-Word   Stemming  Lemmatization
            “studies”  studi ❌ study ✅
             “better”  better ❌    good ✅
           ex o/p-
                 0          upset cant updat facebook text might cri resul...
                 1               dive mani time ball manag save rest go bound
                 2                            whole bodi feel itchi like fire
                 3                                      behav im mad cant see
                 4                                                 whole crew
                                ...                       
                 1048567                              grandma make dinenr mum
                 1048568              midmorn snack time bowl chees noodl yum









'''

nltk.download('stopwords')
stop_words=set(stopwords.words('english'))#contains all common words in english
stemmer=PorterStemmer()
class Pre_Process_Strategies:
    def handle_data(self,df:pd.DataFrame)->pd.DataFrame:
        try:
            #nltk.download('stopwords')
           # stop_words=set(stopwords.words('english'))
           # stemmer=PorterStemmer()
            df.drop(columns='id of the tweet',inplace=True)
            df.drop(columns=['query','user'],inplace=True)
            df.rename(columns={'polarity of tweet�': 'sentiment','date of the tweet': 'date','text of the tweet�': 'text'},inplace=True)
            '''
            It is to be noted that there are no null values and the main col is 'sentimennt' and 'text' so no other 
            '''
            
            df.drop(columns='date',inplace=True) 
           
            df['text']=df['text'].str.lower()
            df['text']=df['text'].str.replace(r'@\w+', '', regex=True)
            def pre_process_further(text):
                text=re.sub(r'http\S+|www\S+','',text)
                text=re.sub(r'[^a-z\s]','',text)
                words=text.split()
                words=[stemmer.stem(w) for w in words if  w not in stop_words]#stemmer.stem(w) for w in words if w not in stop_words
                return ' '.join(words)
            df['text']=df['text'].apply(pre_process_further)
            data=df
            return data
        except Exception as e:
            logging.error(f"Error while cleaning data: {e}")
            raise e  
   
    def split_test_train_and_feature_engineer(self,data:pd.DataFrame)->Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],#should be np.ndarray not no.array
    Annotated[pd.Series, "Y_train"],
    Annotated[pd.Series, "Y_test"]
]:
        try:
            X=data.drop(columns=['sentiment'])
            Y= data['sentiment']
            X_train_df, X_test_df, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, random_state=2)#here 0.2 means 20% of taat will be for test.this methhod is used ot split the data into train and test
            tfid=TfidfVectorizer(max_features=5000,ngram_range=(1,2))#max_features means it tells tfidf to only consider top 5000 words, n_grams=(1,2)means it tells it t conasider eeither single words like good,bad or 2 words like not good,nnot bad etc
            X_train=tfid.fit_transform(X_train_df['text']).toarray()
            X_test=tfid.transform(X_test_df['text']).toarray()
            # create a new filepath and open 'tfidf_vectorizer.pkl' 'wb' meams write in binary mode
            with open('tfidf_vectorizer.pkl','wb') as f:
                pickle.dump(tfid,f)#writes tfid to the file
                print("Saved successfully!")
                           
            return X_train, X_test, Y_train, Y_test
        except Exception as e:
            logging.error(f"Error while dividing data or feature engineering of data: {e}")
            raise e
    
    def clean_deployment_text(self,text:str)->np.ndarray:
        try:
           
            text = text.lower()
            text=re.sub(r'http\S+|www\S+','',text)
            text=re.sub(r'[^a-z\s]','',text)
            text=re.sub(r'@\w+','',text)
            #rb=read binary
            with open('tfidf_vectorizer.pkl','rb')as f:
                tfid=pickle.load(f)
            words=text.split()#splits the string on whitespace by default.
            words=[stemmer.stem(w) for w in words if w not in stop_words]
            cleaned_text = " ".join(words)
            
            pre_processed_words = tfid.transform([cleaned_text]).toarray()
            return pre_processed_words
        except Exception as e:
            logging.error(f'Error in cleaning the raw text for deployment')
            raise e









'''

Bag of words- It means count how many times each word appears in a document.
ex-Example:

Document	Text
D1	"I love dogs"
D2	"I love cats"

Vocabulary = ["I", "love", "dogs", "cats"]

Doc	I	love	dogs	cats
D1	1	1	1	0
D2	1	1	0	1

So each sentence becomes a vector of word counts.

🔸 Problem: common words ("I", "love") appear everywhere — not very informative.

TFIDF-TF-IDF (Term Frequency–Inverse Document Frequency)
make ccommon words less important and rare words important
IDF=log(N/(1+n(t)))
where N = total docs and n(t)=number of docs containing the term
it gives more weight to unique words
But doesnt cqpture synonyms.
We use this in the project because naive bayes and logistic regression do not work properly with word embeddings.
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
fit_transform()- learns vocalblury and IDF weights and then it converts into TF-IDF vectors
transform()-Uses the same vocalblury and TF-IDF vectors from trrainig data aND just transforms the testing data to TF-IDF vectors 
''' 















[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rishilboddula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'\n\nBag of words- It means count how many times each word appears in a document.\nex-Example:\n\nDocument\tText\nD1\t"I love dogs"\nD2\t"I love cats"\n\nVocabulary = ["I", "love", "dogs", "cats"]\n\nDoc\tI\tlove\tdogs\tcats\nD1\t1\t1\t1\t0\nD2\t1\t1\t0\t1\n\nSo each sentence becomes a vector of word counts.\n\n🔸 Problem: common words ("I", "love") appear everywhere — not very informative.\n\nTFIDF-TF-IDF (Term Frequency–Inverse Document Frequency)\nmake ccommon words less important and rare words important\nIDF=log(N/(1+n(t)))\nwhere N = total docs and n(t)=number of docs containing the term\nit gives more weight to unique words\nBut doesnt cqpture synonyms.\nWe use this in the project because naive bayes and logistic regression do not work properly with word embeddings.\n------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
import pandas as pd
from typing import Tuple
import logging
import numpy as np
from typing_extensions import Annotated

def clean_data(df: pd.DataFrame) -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[pd.Series, "Y_train"],
    Annotated[pd.Series, "Y_test"]
]:
    try:
        processor=Pre_Process_Strategies()
        cleaned_df=processor.handle_data(df)
        X_train,X_test,Y_train,Y_test=processor.split_test_train_and_feature_engineer(cleaned_df)
        logging.info("Data cleaning,feature_engineering and division successful")
        return X_train,X_test,Y_train,Y_test

    except Exception as e:


        logging.error(f"Error while cleaning data: {e}")
        raise e


In [8]:
X_train, X_test, y_train, y_test = clean_data(df)

Saved successfully!
Data cleaning,feature_engineering and division successful
Data cleaning,feature_engineering and division successful


In [9]:
X_test.shape

(209715, 5000)

In [14]:
text="I really love this product! It works perfectly."
cleaning_strategies=Pre_Process_Strategies()
cleaned_words=cleaning_strategies.clean_deployment_text(text)


In [15]:
cleaned_words

array([[0., 0., 0., ..., 0., 0., 0.]])

In [16]:
cleaned_words.shape

(1, 5000)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 0 to 1048571
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   polarity of tweet�  1048572 non-null  int64 
 1   id of the tweet     1048572 non-null  int64 
 2   date of the tweet   1048572 non-null  object
 3   query               1048572 non-null  object
 4   user                1048572 non-null  object
 5   text of the tweet�  1048572 non-null  object
dtypes: int64(2), object(4)
memory usage: 48.0+ MB


In [106]:
df.drop(columns='id of the tweet',inplace=True)
df.drop(columns=['query','user'],inplace=True)
df.rename(columns={
    'polarity of tweet�': 'sentiment',
    'date of the tweet': 'date',
    'text of the tweet�': 'text'
},inplace=True)


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 0 to 1048571
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1048572 non-null  int64 
 1   date       1048572 non-null  object
 2   text       1048572 non-null  object
dtypes: int64(1), object(2)
memory usage: 24.0+ MB


In [108]:
import seaborn as sns
import matplotlib.pyplot as plt
df['sentiment']=df['sentiment'].replace({4:1})
'''
In the above line we are telling to replace all 4's with 1. For example if in a dataset there are positive,negativeand we want to replace with ones and 0s,
we can do like df['col_name']=df['col_nale'].replace({positive:1,negative:0})


'''



"\nIn the above line we are telling to replace all 4's with 1. For example if in a dataset there are positive,negativeand we want to replace with ones and 0s,\nwe can do like df['col_name']=df['col_nale'].replace({positive:1,negative:0})\n\n\n"

In [109]:
df['date']=pd.to_datetime(df['date'],errors='coerce')

/Users/rishilboddula/Desktop/MLOPS/Sentiment-Analysis-MLOPS/myenv/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [110]:
df['date'].head(10)

0   2009-04-06 22:19:49
1   2009-04-06 22:19:53
2   2009-04-06 22:19:57
3   2009-04-06 22:19:57
4   2009-04-06 22:20:00
5   2009-04-06 22:20:03
6   2009-04-06 22:20:03
7   2009-04-06 22:20:17
8   2009-04-06 22:20:19
9   2009-04-06 22:20:19
Name: date, dtype: datetime64[ns]

In [111]:
df_daily = df.groupby(df['date'].dt.date)['sentiment'].mean().reset_index()
df_daily.rename(columns={'date':'Date', 'sentiment':'Avg_Sentiment'}, inplace=True)

In [112]:
df_daily=df.groupby(df['date'].dt.date)['sentiment'].mean().reset_index()
#.dt.date extracts only the date (YYYY-MM-DD) from a datetime column, removing the time.
df_daily.rename(columns={'date':'Date','sentiment':'Avg_sentiment'},inplace=True)

In [113]:
df_daily

,Date,Avg_sentiment
0,2009-04-06,0.576281
1,2009-04-07,0.587026
2,2009-04-17,0.596628
3,2009-04-18,0.584444
4,2009-04-19,0.584004
5,2009-04-20,0.576544
6,2009-04-21,0.591941
7,2009-05-01,0.578279
8,2009-05-02,0.584530
9,2009-05-03,0.581722


In [114]:
plt.figure(figsize=(12,6))
sns.lineplot(x='Date', y='Avg_Sentiment', data=df_daily)
plt.xlabel('Date')
plt.ylabel('Average Sentiment (0 = Negative, 1 = Positive)')
plt.title('Sentiment Trend Over Time')
plt.xticks(rotation=45)  # rotate dates to see them clearly
plt.show()

ValueError: Could not interpret value `Avg_Sentiment` for `y`. An entry with this name does not appear in `data`.

<Figure size 1200x600 with 0 Axes>

In [ ]:
df['text']=df['text'].str.lower()

In [ ]:
df.head()

,sentiment,date,text
0,0,2009-04-06 22:19:49,is upset that he can't update his facebook by ...
1,0,2009-04-06 22:19:53,@kenichan i dived many times for the ball. man...
2,0,2009-04-06 22:19:57,my whole body feels itchy and like its on fire
3,0,2009-04-06 22:19:57,"@nationwideclass no, it's not behaving at all...."
4,0,2009-04-06 22:20:00,@kwesidei not the whole crew


In [ ]:
df['text']=df['text'].str.replace(r'@\w+', '', regex=True)

In [ ]:
df.head()

,sentiment,date,text
0,0,2009-04-06 22:19:49,is upset that he can't update his facebook by ...
1,0,2009-04-06 22:19:53,i dived many times for the ball. managed to s...
2,0,2009-04-06 22:19:57,my whole body feels itchy and like its on fire
3,0,2009-04-06 22:19:57,"no, it's not behaving at all. i'm mad. why am..."
4,0,2009-04-06 22:20:00,not the whole crew


In [ ]:
df.drop(columns='date',inplace=True)

In [115]:
df.head()

,sentiment,date,text
0,0,2009-04-06 22:19:49,is upset that he can't update his Facebook by ...
1,0,2009-04-06 22:19:53,@Kenichan I dived many times for the ball. Man...
2,0,2009-04-06 22:19:57,my whole body feels itchy and like its on fire
3,0,2009-04-06 22:19:57,"@nationwideclass no, it's not behaving at all...."
4,0,2009-04-06 22:20:00,@Kwesidei not the whole crew


In [116]:
print(df.columns)


Index(['sentiment', 'date', 'text'], dtype='object')


In [ ]:
df['text']=df['text'].str.lower()

In [117]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))
stemmer=PorterStemmer()
lemmetizer=WordNetLemmatizer()
def pre_prrocess(text):
    text=re.sub(r'http\S+|www\S+','',text)#remove the urls
    text=re.sub(r'[^a-z\s]','',text)
    words=text.split()#puts a sentence of words in a list
    words=[lemmetizer.lemmatize(w) for w in words if w not in stop_words]
    return ' '.join(words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rishilboddula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rishilboddula/nltk_data...


In [ ]:
df['text']=df['text'].apply(pre_prrocess)

In [ ]:
df['text']

0          upset cant updat facebook text might cri resul...
1               dive mani time ball manag save rest go bound
2                            whole bodi feel itchi like fire
3                                      behav im mad cant see
4                                                 whole crew
                                 ...                        
1048567                              grandma make dinenr mum
1048568              midmorn snack time bowl chees noodl yum
1048569                say like terminus movi come like word
1048570                                 im great thaank wbuu
1048571                           cant wait til date weekend
Name: text, Length: 1048572, dtype: object

In [ ]:
df

,sentiment,text
0,0,upset cant updat facebook text might cri resul...
1,0,dive mani time ball manag save rest go bound
2,0,whole bodi feel itchi like fire
3,0,behav im mad cant see
4,0,whole crew
...,...,...
1048567,1,grandma make dinenr mum
1048568,1,midmorn snack time bowl chees noodl yum
1048569,1,say like termini movi come like word
1048570,1,im great thaank wbuu


In [ ]:
from sklearn.model_selection import train_test_split